# Scaled vertical tectonics from backward vs computed elevations

In [ ]:
import numpy as np
from scipy import spatial
from scipy import ndimage
from script import readOutput as output

import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 7
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size
matplotlib.rc('font', size=6)

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

Assuming uniform vertical displacements evolution, the scaling factor is growing from 0.2 to 1. over a 5 Ma period when considering a 1Ma interval.

In [ ]:
it = 20
factor = 0.6

backfile = 'data/backward-elev15-20.npz'
backstep = 'z'+str(it-1)

modelinput = 'inputs/model'+str(it)+'Ma.yml'
outNb = 100 # Final time step for the model (last step in Paraview...)

sigma = 10 # Gaussian filter on tectonic grid
tecfile = 'input'+str(it)+'/vtec'+str(it)+'Ma'

res = 0.1
dt = 1.e6

Main loop for mapping the tectonic differences over a 0.1 degree structured map

In [ ]:
out = output.readOutput(filename=modelinput, step=outNb, uplift=False)
out.buildLonLatMesh(res=res, nghb=3)

shape = out.z.shape
lon = np.linspace(0.0, 360, shape[1])
lat = np.linspace(0.0, 180, shape[0])
glon, glat = np.meshgrid(lon, lat)
glonlat = np.dstack([glon.flatten(), glat.flatten()])[0]

backMesh = np.load(backfile)
back_elev = backMesh[backstep]

tecto = factor * (back_elev - out.z.flatten())
tecto = ndimage.gaussian_filter(tecto,sigma)

In [ ]:
# import meshio
# vtkfile = 'model19Ma.vtk'
# vis_mesh = meshio.Mesh(out.vertices, {'triangle': out.cells}, 
#                                    point_data={"Z":out.elev, "ED":out.erodep})
# meshio.write(vtkfile, vis_mesh)

Check the scaled tectonic differences...

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
ax.imshow(np.flipud(tecto.reshape(shape)), extent=(-180, 180, -90, 90), vmin=-500, vmax=500, cmap=cm.RdBu) 
ax.set(xlabel='Longitude', ylabel='Latitude', yticks=np.arange(-90,120,30), xticks=np.arange(-180,180,30))
ax.minorticks_on()

Map the tectonic displacement on the unstructured mesh

In [ ]:
tree = spatial.cKDTree(glonlat, leafsize=10)

distances, indices = tree.query(out.lonlat, k=3)
onIDs = np.where(distances[:, 0] == 0)[0]

# Inverse weighting distance...
weights = np.divide(
    1.0, distances ** 2, out=np.zeros_like(distances), where=distances != 0
)
sumweights = np.sum(weights, axis=1)
onIDs = np.where(sumweights == 0)[0]
sumweights[sumweights == 0] = 1.0e-4

tec = tecto.flatten()/dt
uplift = np.sum(weights * tec[indices][:, :], axis=1) / sumweights

if len(onIDs) > 0:
    uplift[onIDs] = tec[indices[onIDs, 0]]
    
np.savez_compressed(tecfile, z=uplift)

For checking in Paraview

In [ ]:
# import meshio

# mesh_struct = np.load('input20/20Ma.npz')

# print("Field names: {}".format(mesh_struct.files))

# ngbIDs = mesh_struct['n']
# vertices = mesh_struct['v']
# cells = mesh_struct['c']
# elev = mesh_struct['z']

# checkvtk = "check.vtk"
# vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, point_data={"z":elev, "vtec":uplift})
# meshio.write(checkvtk, vis_mesh)